<h1>Gather and Clean Data</h1>

<h4>Import Dependencies</h4>

In [18]:
import requests 
import json
from pprint import pprint
import os
import pandas as pd
import csv

<h4>Read in GDP csv data</h4>

In [20]:
# Import GDP data
path_gdp = os.path.join("Resources", "GDP_1991_to_2016.csv")
gdp_df = pd.read_csv(path_gdp, encoding = "ISO-8859-1", engine='python')
gdp_df.head()

,ï»¿Country Name,Country Code,Indicator Name,Indicator Code,1991,1992,1993,1994,1995,1996,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,8.721387e+08,9.584632e+08,1.082980e+09,1.245688e+09,1.320475e+09,1.379961e+09,...,2.615084e+09,2.745251e+09,2.498883e+09,2.390503e+09,2.549721e+09,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09
1,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,9.747880e+09,1.010923e+10,1.243909e+10,1.585657e+10,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.936264e+10
2,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,1.060378e+10,8.307811e+09,5.768720e+09,4.438321e+09,5.538749e+09,7.526447e+09,...,6.526645e+10,8.853861e+10,7.030716e+10,8.379950e+10,1.117900e+11,1.280530e+11,1.367100e+11,1.457120e+11,1.161940e+11,1.011240e+11
3,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,1.099559e+09,6.521750e+08,1.185315e+09,1.880952e+09,2.392765e+09,3.199643e+09,...,1.067732e+10,1.288135e+10,1.204422e+10,1.192693e+10,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10
4,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,1.106929e+09,1.210014e+09,1.007026e+09,1.017549e+09,1.178739e+09,1.223945e+09,...,3.952601e+09,4.085631e+09,3.674410e+09,3.449967e+09,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09


In [21]:
# Import Weather data
# Import temperature data
path_temp = os.path.join("Resources", "Temp_Data.csv")
temp_df = pd.read_csv(path_temp, encoding = "ISO-8859-1", engine='python')

# rename columns
temp_df = temp_df.rename(columns={'Temp_C': 'Temperature_Celsius', ' Year': 'Year', ' Statistics': 'Statistics', ' Country': 'Country', ' ISO3': 'ISO3'})
temp_df.columns

# # # Convert Temps from C to F
temp_df['Temp_Farenheit'] = ((temp_df['Temperature_Celsius']*(9/5)+32))
temp_df.head()

#  Get average temp per yr for eeach country
meanTemp_df = temp_df.groupby(['Year','Country', 'ISO3'])['Temp_Farenheit'].mean()
meanTemp_df

# # Import rainfall data
rain_df = pd.read_csv('Resources/Precip_Data.csv')
rain_df.columns
# rename columns
rain_df = rain_df.rename(columns={'Rainfall - (MM)': 'Rainfall_mm', ' Year': 'Year', ' Statistics': 'Statistics', ' Country': 'Country', ' ISO3': 'ISO3'})
rain_df.columns

#  Get average temp per yr for eeach country
meanRain_df = rain_df.groupby(['Year','Country', 'ISO3'])['Rainfall_mm'].mean()
meanRain_df

# # # # Combine temp and weather data into a single dataframe
weather_df = pd.merge(meanTemp_df, meanRain_df, how='left', on=['ISO3', 'Year', 'Country'])
weather_df = weather_df.sort_values(by=['ISO3', 'Year'])
weather_df = weather_df.reset_index()
weather_df = weather_df.dropna(how='any')
weather_df

weather_df.to_csv('Resources/weather_df.csv')

<h4>Read in Country ISO3 Codes from API source and store it in a dictionary</h4>

In [22]:
import requests

code_dict = {}
my_request_thing = [x.strip() for x in weather_df['Country'].unique().tolist()]
for name in my_request_thing:
    url = f"https://restcountries.eu/rest/v2/name/{name}?fullText=true"
    response = requests.get(url)
    print(name, end='\r')
    try:
        code_dict[name] = response.json()[0]['alpha3Code']
    except KeyError:
        code_dict[name] = None
code_dict

{'Afghanistan': 'AFG',
 'Angola': 'AGO',
 'Albania': 'ALB',
 'Andorra': 'AND',
 'United Arab Emirates': 'ARE',
 'Argentina': 'ARG',
 'Armenia': 'ARM',
 'Antigua and Barbuda': 'ATG',
 'Australia': 'AUS',
 'Austria': 'AUT',
 'Azerbaijan': 'AZE',
 'Burundi': 'BDI',
 'Belgium': 'BEL',
 'Benin': 'BEN',
 'Burkina Faso': 'BFA',
 'Bangladesh': 'BGD',
 'Bulgaria': 'BGR',
 'Bahrain': 'BHR',
 'Bahamas': 'BHS',
 'Bosnia and Herzegovina': 'BIH',
 'Belarus': 'BLR',
 'Belize': 'BLZ',
 'Bolivia': None,
 'Brazil': 'BRA',
 'Barbados': 'BRB',
 'Brunei': None,
 'Bhutan': 'BTN',
 'Botswana': 'BWA',
 'Central African Republic': 'CAF',
 'Canada': 'CAN',
 'Switzerland': 'CHE',
 'Chile': 'CHL',
 'China': 'CHN',
 "Cote d'Ivoire": 'CIV',
 'Cameroon': 'CMR',
 'Congo (Democratic Republic of the)': 'COD',
 'Congo (Republic of the)': None,
 'Colombia': 'COL',
 'Comoros': 'COM',
 'Cape Verde': None,
 'Costa Rica': 'CRI',
 'Cuba': 'CUB',
 'Cyprus': 'CYP',
 'Czech Republic': 'CZE',
 'Germany': 'DEU',
 'Djibouti': 'DJI'